In [242]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import tree
import os
import torch 
from decimal import Decimal
from torch.utils.data import Dataset, DataLoader,TensorDataset
import numpy as np
import math
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import matplotlib.pyplot as plt
import multiprocessing
import glob
import os
from scipy import linalg
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from torch.autograd import Variable 

In [243]:
df=pd.read_csv('data_without_ioa.csv')

In [244]:
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [245]:
df['label'].value_counts()


0     1644290
8       43831
7       43403
11      37363
10      27504
9       16542
Name: label, dtype: int64

In [246]:
#majority under sampling
# df
# data=df[df['label']!=0]
# data_s = resample(df[df['label']==0], replace=False,n_samples=10000,random_state=123)
# df=pd.concat([data,data_s])

In [247]:
#All sampling
data_train_sampled=pd.DataFrame([],columns=df.columns.values,dtype=float)
sampling=30000
for i in df['label'].unique():
    if df[df['label']==i].shape[0]>sampling:
        
        data_s = resample(df[df['label']==i], replace=False,n_samples=sampling,random_state=123)
        data_train_sampled=pd.concat([data_train_sampled,data_s])
    else:
        data_s = resample(df[df['label']==i], replace=True,n_samples=sampling,random_state=123) 
        data_train_sampled=pd.concat([data_train_sampled,data_s])

In [248]:
# X,y = df.iloc[:, :-1], df.iloc[:, [-1]]
X, y = data_train_sampled.iloc[:, :-1], data_train_sampled.iloc[:, [-1]]



In [249]:
y['label'].value_counts()


0.0     30000
7.0     30000
8.0     30000
9.0     30000
10.0    30000
11.0    30000
Name: label, dtype: int64

In [250]:
mm = MinMaxScaler()

X_mm = mm.fit_transform(X) 


df_data=torch.Tensor(X_mm)
df_label=y.values[:,0]

# y=torch.Tensor(y.values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = df_label.reshape(len(df_label), 1)
encoded_label = onehot_encoder.fit_transform(integer_encoded)
encoded_label=torch.Tensor(encoded_label)




In [251]:
X_train, X_test, y_train, y_test = train_test_split(df_data, encoded_label, test_size=0.3,random_state=109) # 70% training and 30% test


In [252]:
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, shuffle=True, batch_size=train_data.tensors[0].shape[0])
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0])

print("Training data batches:")
for X, y in train_loader:
    print(X.shape, y.shape)
    
print("\nTest data batches:")
for X, y in test_loader:
    print(X.shape, y.shape)

Training data batches:
torch.Size([126000, 15]) torch.Size([126000, 6])

Test data batches:
torch.Size([54000, 15]) torch.Size([54000, 6])


In [253]:

#Option 1
in_f=train_data.tensors[0].shape[1]
out_f=train_data.tensors[1].shape[1]

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(in_features=in_f, out_features=128)
        self.hidden_1 = nn.Linear(in_features=128, out_features=64)
        self.hidden_12 = nn.Linear(in_features=64, out_features=32)

        self.output = nn.Linear(in_features=32, out_features=out_f)
        
        
        
    def forward(self, x):
        x = F.sigmoid(self.input(x))
        x = F.sigmoid(self.hidden_1(x))
        x = F.sigmoid(self.hidden_12(x))

    
        return self.output(x)
    
    
model = Net()
print(model)

Net(
  (input): Linear(in_features=15, out_features=128, bias=True)
  (hidden_1): Linear(in_features=128, out_features=64, bias=True)
  (hidden_12): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=6, bias=True)
)


In [254]:
#Option 2

class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [255]:
model = MulticlassClassification(num_feature = in_f, num_class=out_f)

In [256]:
num_epochs = 300

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

for epoch in range(num_epochs):
    for X, y in train_loader:
        preds = model(X)
        preds=torch.softmax(preds,axis=1)
        
        loss = loss_function(preds, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        y_labels=torch.argmax(y,axis=1)
        pred_labels=torch.argmax(preds,axis=1)
        
        acc=100 * torch.mean((pred_labels == y_labels).float()).item()
            
    

    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss.item(), acc))
    
    X , y = next(iter(test_loader))
    preds=torch.softmax(model(X),axis=1)
    y_labels=torch.argmax(y,axis=1)
    pred_labels=torch.argmax(preds,axis=1)
    testacc=100 * torch.mean((pred_labels == y_labels).float()).item()
    print('Test Accuracy:', testacc)
    print(metrics.classification_report(y_labels, pred_labels))


Epoch 1/300, Loss: 1.794, Accuracy: 17.001
Test Accuracy: 39.790740609169006
              precision    recall  f1-score   support

           0       0.20      0.24      0.22      8950
           1       0.64      0.45      0.53      9004
           2       0.30      0.06      0.11      9055
           3       0.41      0.52      0.46      8993
           4       0.39      0.27      0.32      9030
           5       0.44      0.84      0.57      8968

    accuracy                           0.40     54000
   macro avg       0.40      0.40      0.37     54000
weighted avg       0.40      0.40      0.37     54000

Epoch 2/300, Loss: 1.708, Accuracy: 39.854
Test Accuracy: 50.777775049209595
              precision    recall  f1-score   support

           0       0.25      0.16      0.19      8950
           1       0.67      0.69      0.68      9004
           2       0.55      0.08      0.14      9055
           3       0.41      0.63      0.50      8993
           4       0.52      0.5

/Users/hadeerteryak/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hadeerteryak/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hadeerteryak/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 9/300, Loss: 1.425, Accuracy: 64.410
Test Accuracy: 65.22963047027588
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      8950
           1       0.78      0.75      0.77      9004
           2       0.53      0.55      0.54      9055
           3       0.84      0.80      0.82      8993
           4       0.48      0.80      0.60      9030
           5       0.72      1.00      0.84      8968

    accuracy                           0.65     54000
   macro avg       0.56      0.65      0.60     54000
weighted avg       0.56      0.65      0.60     54000



/Users/hadeerteryak/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hadeerteryak/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hadeerteryak/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 10/300, Loss: 1.413, Accuracy: 65.240
Test Accuracy: 66.09259247779846
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      8950
           1       0.79      0.75      0.77      9004
           2       0.55      0.55      0.55      9055
           3       0.86      0.80      0.83      8993
           4       0.50      0.87      0.63      9030
           5       0.73      1.00      0.84      8968

    accuracy                           0.66     54000
   macro avg       0.57      0.66      0.60     54000
weighted avg       0.57      0.66      0.60     54000



...

Epoch 244/300, Loss: 1.058, Accuracy: 98.669
Test Accuracy: 98.56851696968079
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      8950
           1       0.99      1.00      1.00      9004
           2       0.99      1.00      1.00      9055
           3       0.98      1.00      0.99      8993
           4       0.99      1.00      0.99      9030
           5       0.97      1.00      0.98      8968

    accuracy                           0.99     54000
   macro avg       0.99      0.99      0.99     54000
weighted avg       0.99      0.99      0.99     54000

Epoch 245/300, Loss: 1.058, Accuracy: 98.660
Test Accuracy: 98.57962727546692
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      8950
           1       0.99      1.00      1.00      9004
           2       0.99      1.00      1.00      9055
           3       0.98      1.00      0.99      8993
           4       0.99      1

Epoch 258/300, Loss: 1.057, Accuracy: 98.730
Test Accuracy: 98.63333106040955
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      8950
           1       0.99      1.00      1.00      9004
           2       0.99      1.00      1.00      9055
           3       0.98      1.00      0.99      8993
           4       0.99      1.00      0.99      9030
           5       0.97      1.00      0.98      8968

    accuracy                           0.99     54000
   macro avg       0.99      0.99      0.99     54000
weighted avg       0.99      0.99      0.99     54000

Epoch 259/300, Loss: 1.057, Accuracy: 98.742
Test Accuracy: 98.67963194847107
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      8950
           1       0.99      1.00      1.00      9004
           2       0.99      1.00      1.00      9055
           3       0.98      1.00      0.99      8993
           4       0.99      1

Epoch 272/300, Loss: 1.057, Accuracy: 98.789
Test Accuracy: 98.67592453956604
              precision    recall  f1-score   support

           0       1.00      0.93      0.96      8950
           1       0.99      1.00      1.00      9004
           2       0.99      1.00      1.00      9055
           3       0.98      1.00      0.99      8993
           4       0.99      1.00      0.99      9030
           5       0.97      1.00      0.98      8968

    accuracy                           0.99     54000
   macro avg       0.99      0.99      0.99     54000
weighted avg       0.99      0.99      0.99     54000

Epoch 273/300, Loss: 1.057, Accuracy: 98.788
Test Accuracy: 98.69629740715027
              precision    recall  f1-score   support

           0       1.00      0.93      0.96      8950
           1       0.99      1.00      1.00      9004
           2       0.99      1.00      1.00      9055
           3       0.98      1.00      0.99      8993
           4       0.99      1

Epoch 286/300, Loss: 1.056, Accuracy: 98.851
Test Accuracy: 98.77036809921265
              precision    recall  f1-score   support

           0       1.00      0.93      0.96      8950
           1       0.99      1.00      1.00      9004
           2       1.00      1.00      1.00      9055
           3       0.98      1.00      0.99      8993
           4       0.99      1.00      1.00      9030
           5       0.97      1.00      0.98      8968

    accuracy                           0.99     54000
   macro avg       0.99      0.99      0.99     54000
weighted avg       0.99      0.99      0.99     54000

Epoch 287/300, Loss: 1.056, Accuracy: 98.813
Test Accuracy: 98.72221946716309
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      8950
           1       0.99      1.00      1.00      9004
           2       0.99      1.00      1.00      9055
           3       0.98      1.00      0.99      8993
           4       0.99      1

Epoch 300/300, Loss: 1.056, Accuracy: 98.872
Test Accuracy: 98.7999975681305
              precision    recall  f1-score   support

           0       1.00      0.93      0.96      8950
           1       0.99      1.00      1.00      9004
           2       1.00      1.00      1.00      9055
           3       0.98      1.00      0.99      8993
           4       0.99      1.00      1.00      9030
           5       0.97      1.00      0.98      8968

    accuracy                           0.99     54000
   macro avg       0.99      0.99      0.99     54000
weighted avg       0.99      0.99      0.99     54000



In [257]:
#0 7 8 9 10 11 --> 0 1 2 3 4 5